In [1]:
import numpy as np
import pandas as pd

np.set_printoptions(precision = 3, suppress = True)

import tensorflow as tf
from tensorflow.keras import layers

In [2]:
# loading csv data
df = pd.read_csv("/Users/olivia.wilcox13/Documents/PQHS 416/pqhs416_dataset.csv")



In [3]:
y = df['DIAGNOSIS'].values
y.shape

(80,)

In [4]:
x = df[['MICROSCOPY','IMAGING RESULTS','IMMUNOHISTOCHEMISTRY','BRAIN LOCALIZATION']].values
x.shape

(80, 4)

In [5]:
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

#splitting into training and testing
(x_train, x_test, y_train, y_test) = train_test_split(x,y, test_size = 0.4)
x_train.shape

(48, 4)

In [6]:
#making training dataframe
df1 =pd.DataFrame(x_train)
df1=df1.rename(columns={0:'microscopy', 1:'imaging results', 2:'immunohistochemistry', 3:'brain localization'})
df2 = pd.DataFrame(y_train)
df2 = df2.rename(columns = {0:'diagnosis'})
df_train = pd.concat([df2, df1], axis=1)
df_train.head()

,diagnosis,microscopy,imaging results,immunohistochemistry,brain localization
0,Mild Malformation of Cortical Development Type II,Ectopic neurons in white matter,Hyperintense right amygdala,MAP2-positive,Right temporal lobe
1,Focal Cortical Dysplasia Type Ib,Infiltrating macrophage,MRI negative,MAP2-positive,Right frontal lobe
2,Focal Cortical Dysplasia Type Ib,Laminar cortical architecture disturbed,Focal Cortical Dysplasia in right temporo-occi...,GFAP-positive,Right temporo-occipital
3,Hippocampal sclerosis type I,No neuronal dysmorphia,Right hippocampal sclerosis,MAP2-negative,Right mesial temporal lobe
4,Hippocampal sclerosis type I,Horizontal dyslamination,Global cortical atrophy,NeuN-positive,Temporal lobe


In [7]:
#making testing dataframe
df3 = pd.DataFrame(x_test)
df3 = df3.rename(columns={0:'microscopy', 1:'imaging results', 2:'immunohistochemistry', 3:'brain localization'})
df4 = pd.DataFrame(y_test)
df4 = df4.rename(columns={0:'diagnosis'})
df_test=pd.concat([df4, df3], axis = 1)
df_test.head()

,diagnosis,microscopy,imaging results,immunohistochemistry,brain localization
0,Focal Cortical Dysplasia TypeIIb,Cortical dyslamination,Left parieto-temporal lesion,MAP2-positive,Left parieto-temporal region
1,Mild Malformation of Cortical Development,No vertical dyslamination,Right frontopolar dysplasia,MAP2-positive,Frontal lobe
2,Oligodendroglial hyperplasia,No neuronal dysmorphia,Right frontal lobe,MAP2-positive,Right frontal lobe
3,Mild Malformation of Cortical Development Type II,Blurred gray white matter,Encephalocele,NeuN-negative,Right temporal lobe
4,Mild Malformation of Cortical Development Type II,Blurred gray white matter,Negative 3Tesla MRI,MAP2-positive,Right frontal lobe


In [8]:
max_features = 1000
max_len = 200
vectorize_layer = tf.keras.layers.TextVectorization(
max_tokens = max_features, output_mode = 'int',
output_sequence_length=max_len)

2022-04-19 16:40:05.572230: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
# text vectorization for training data
# text vectorization for output (diagnosis)
text_vectorizer = layers.TextVectorization(output_mode="multi_hot", ngrams=3)
text_vectorizer.adapt(df_train['diagnosis'])
train_d = text_vectorizer(df_train['diagnosis'])

# text vectorization for microscopy
text_vectorizer = layers.TextVectorization(output_mode="multi_hot", ngrams=3)
text_vectorizer.adapt(df_train['microscopy'])
train_m = text_vectorizer(df_train['microscopy'])

# text vectorization for imaging results
text_vectorizer = layers.TextVectorization(output_mode="multi_hot", ngrams=3)
text_vectorizer.adapt(df_train['imaging results'])
train_i = text_vectorizer(df_train['imaging results'])


# text vectorization for immunohistochemistry
text_vectorizer = layers.TextVectorization(output_mode="multi_hot", ngrams=3)
text_vectorizer.adapt(df_train['immunohistochemistry'])
train_immuno = text_vectorizer(df_train['immunohistochemistry'])


# text vectorization for brain localization
text_vectorizer = layers.TextVectorization(output_mode="multi_hot", ngrams=3)
text_vectorizer.adapt(df_train['brain localization'])
train_bl = text_vectorizer(df_train['brain localization'])


# combining text vectorizations into one training 
x_train = tf.concat([train_m, train_i, train_immuno, train_bl], axis = 1)
print (x_train.shape)

(48, 505)


In [10]:
# text vectorization for testing data
# text vectorization for output(diagnosis)
text_vectorizer = layers.TextVectorization(output_mode="multi_hot", ngrams=3)
text_vectorizer.adapt(df_test['diagnosis'])
test_d = text_vectorizer(df_test['diagnosis'])

# text vectorization for microscopy
text_vectorizer = layers.TextVectorization(output_mode="multi_hot", ngrams=3)
text_vectorizer.adapt(df_test['microscopy'])
test_m = text_vectorizer(df_test['microscopy'])

# text vectorization for imaging results
text_vectorizer = layers.TextVectorization(output_mode="multi_hot", ngrams=3)
text_vectorizer.adapt(df_test['imaging results'])
test_i = text_vectorizer(df_test['imaging results'])

# text vectorization for immunohistochemistry
text_vectorizer = layers.TextVectorization(output_mode="multi_hot", ngrams=3)
text_vectorizer.adapt(df_test['immunohistochemistry'])
test_immuno = text_vectorizer(df_test['immunohistochemistry'])

# text vectorization for brain localization
text_vectorizer = layers.TextVectorization(output_mode="multi_hot", ngrams=3)
text_vectorizer.adapt(df_test['brain localization'])
test_bl = text_vectorizer(df_test['brain localization'])

# combing text vectorizations into testing data
x_test = tf.concat([test_d, test_m, test_i, test_immuno, test_bl], axis = 1)

In [11]:
from tensorflow.keras.utils import to_categorical
y_train = to_categorical (train_d)